In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
import geopy.distance
import itertools

from sklearn.mixture import BayesianGaussianMixture
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.svm import SVR,SVC

from sklearn.ensemble import RandomForestRegressor, IsolationForest,ExtraTreesClassifier,RandomForestClassifier
from sklearn.preprocessing import StandardScaler,KBinsDiscretizer,LabelEncoder,MinMaxScaler
from sklearn.metrics import mean_squared_error,f1_score,confusion_matrix,log_loss
from sklearn.kernel_approximation import Nystroem
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier
from sklearn.decomposition import PCA,NMF

from catboost import Pool, cv,CatBoostClassifier,CatBoostRegressor

from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, concatenate,Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
import tensorflow_addons as tfa

import lightgbm as lgb

import xgboost as xgb
from xgboost import XGBClassifier,XGBRegressor,DMatrix,plot_tree

from imblearn.over_sampling import RandomOverSampler,SMOTE, ADASYN

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submission_df = pd.read_csv('sample_submission.csv')

# Feature engineering

## Shortened eda notebook

In [3]:
def extract_time(feature='date'):
    def f(time_stamp): 
        date,time = time_stamp.strip().split()
        date = list(map(int, date.split('/')))
        time = list(map(int, time.split(':')))
        if feature == 'date':
            return date[1]
        if feature == 'month':
            return date[0]
        if feature == 'year':
            return date[2]
        if feature == 'hour':
            return time[0]
        if feature == 'minute':
            return time[1]
    return f
        

In [4]:
def get_combined_dataset(cols, train=train_df, test=test_df):
    tmp_1 = train.loc[:,cols]
    tmp_1['dataset'] = 'train'

    tmp_2 = test.loc[:,cols]
    tmp_2['dataset'] = 'test'
    return tmp_1.append(tmp_2)
    

In [5]:
data = train_df.dropna().drop(['tripid'],axis=1)

In [6]:
data['pickup_month'] = data['pickup_time'].map(extract_time('month'))
data['pickup_date'] = data['pickup_time'].map(extract_time('date'))
data['pickup_year'] = data['pickup_time'].map(extract_time('year'))
data['pickup_hour'] = data['pickup_time'].map(extract_time('hour'))
data['pickup_minute'] = data['pickup_time'].map(extract_time('minute'))

data['drop_month'] = data['drop_time'].map(extract_time('month'))
data['drop_date'] = data['drop_time'].map(extract_time('date'))
data['drop_year'] = data['drop_time'].map(extract_time('year'))
data['drop_hour'] = data['drop_time'].map(extract_time('hour'))
data['drop_minute'] = data['drop_time'].map(extract_time('minute'))

In [7]:
test_df['pickup_month'] = test_df['pickup_time'].map(extract_time('month'))
test_df['pickup_date'] = test_df['pickup_time'].map(extract_time('date'))
test_df['pickup_year'] = test_df['pickup_time'].map(extract_time('year'))
test_df['pickup_hour'] = test_df['pickup_time'].map(extract_time('hour'))
test_df['pickup_minute'] = test_df['pickup_time'].map(extract_time('minute'))

test_df['drop_month'] = test_df['drop_time'].map(extract_time('month'))
test_df['drop_date'] = test_df['drop_time'].map(extract_time('date'))
test_df['drop_year'] = test_df['drop_time'].map(extract_time('year'))
test_df['drop_hour'] = test_df['drop_time'].map(extract_time('hour'))
test_df['drop_minute'] = test_df['drop_time'].map(extract_time('minute'))

In [8]:
data = data[data['drop_lat'] < 30]

In [9]:
def is_more_than_one_day(row):
    return 1 if row['pickup_date'] != row['drop_date'] else 0

In [10]:
data['is_more_than_one_day'] = data.apply(is_more_than_one_day,axis=1)
test_df['is_more_than_one_day'] = test_df.apply(is_more_than_one_day,axis=1)

In [11]:
cols = ['pick_lat','pick_lon']
comb_data = get_combined_dataset(cols, train=data)

gmm_pick = BayesianGaussianMixture(n_components=3)
gmm_pick.fit(comb_data[cols].values)

data['pick_cluster'] = gmm_pick.predict(data[cols].values)
test_df['pick_cluster'] = gmm_pick.predict(test_df[cols].values)

In [12]:
cols = ['drop_lat','drop_lon']
comb_data = get_combined_dataset(cols, train=data)

gmm_drop = BayesianGaussianMixture(n_components=3,max_iter=1000)
gmm_drop.fit(comb_data[cols].values)

data['drop_cluster'] = gmm_drop.predict(data[cols].values)
test_df['drop_cluster'] = gmm_drop.predict(test_df[cols].values)

In [13]:
def calculate_trip_distance(row):
    coords_1 = (row['pick_lat'],row['pick_lon'])
    coords_2 = (row['drop_lat'],row['drop_lon'])
    return geopy.distance.geodesic(coords_1, coords_2).km

In [14]:
data['distance_km'] = data.apply(calculate_trip_distance,axis=1).clip(0,100)
test_df['distance_km'] = test_df.apply(calculate_trip_distance,axis=1).clip(0,100)

In [15]:
def fare_per_distance(row):
    return row['fare'] / (row['distance_km']+0.01)

In [16]:
data['fare_per_km'] = data.apply(fare_per_distance,axis=1)
test_df['fare_per_km'] = test_df.apply(fare_per_distance,axis=1)

In [17]:
def time_slot(row,by='pickup'):
    hour = row[f'{by}_hour']
    if 7 <= hour <= 9:
        return 1
    if 12 <= hour <= 2:
        return 2
    if 4 <= hour <= 6:
        return 3
    return 0

In [18]:
data['pickup_timeslot'] = data.apply(time_slot,axis=1)
test_df['pickup_timeslot'] = test_df.apply(time_slot,axis=1)

In [19]:
def day_of_week(row,by='pickup'):
    date = row[f'{by}_date']
    month = row[f'{by}_month']
    year = row[f'{by}_year']
    d = datetime.datetime(year,month,date).weekday()
    return d

In [20]:
data['day_of_week'] = data.apply(day_of_week,axis=1)
test_df['day_of_week'] = test_df.apply(day_of_week,axis=1)

In [21]:
def is_weekday(row,by='pickup'):
    date = row['day_of_week']
    return 1 if date < 5 else 0

In [22]:
data['is_weekday'] = data.apply(is_weekday,axis=1)
test_df['is_weekday'] = test_df.apply(is_weekday,axis=1)

In [23]:
def cal_time_difference(row):
    pickup_date = row['pickup_date']
    pickup_month = row['pickup_month']
    pickup_year = row['pickup_year']
    pickup_hour = row['pickup_hour']
    pickup_minute = row['pickup_minute']
    pickup_time = datetime.datetime(pickup_year, pickup_month, pickup_date, pickup_hour, pickup_minute)
    
    drop_date = row['drop_date']
    drop_month = row['drop_month']
    drop_year = row['drop_year']
    drop_hour = row['drop_hour']
    drop_minute = row['drop_minute']
    drop_time = datetime.datetime(drop_year, drop_month, drop_date, drop_hour, drop_minute)
    
    delta = drop_time - pickup_time
    return delta.seconds - row['duration']

In [24]:
data['cal_time_difference'] = data.apply(cal_time_difference,axis=1)
test_df['cal_time_difference'] = test_df.apply(cal_time_difference,axis=1)

In [25]:
def encode_label(label):
    if label == 'correct':
        return 1
    elif label == 'incorrect':
        return 0
    else:
        return label

In [26]:
data['label'] = data['label'].map(encode_label)

In [27]:
columns_to_save = [
    'additional_fare', 
    'duration', 
    'meter_waiting', 
    'meter_waiting_fare',
    'meter_waiting_till_pickup', 
    'fare',
    'pickup_date', 
    'pickup_hour', 
    'pickup_minute',
    'drop_date', 
    'drop_hour', 
    'drop_minute',
    'pick_cluster',
    'is_more_than_one_day',
    'distance_km',
    'fare_per_km',
    'pickup_timeslot',
    'day_of_week',
    'is_weekday',
    'cal_time_difference']
data.loc[:, columns_to_save+['label']].to_csv('train_df.csv',index=False)
test_df.loc[:, columns_to_save].to_csv('test_df.csv',index=False)

## Shortened anomaly_detection notebook

In [28]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')
submission_df = pd.read_csv('sample_submission.csv')

data = train_df[train_df['label'] == 1].dropna()

In [29]:
def anomaly_pred(col, train_df=train_df, test_df=test_df, folds=3):
    labels = train_df['label'].values
    X = train_df[col].values

    X_train_df = train_df[col].values
    X_test_df = test_df[col].values
    
    skf = StratifiedKFold(n_splits=3)

    validation_scores = []
    models = []

    train_preds = np.zeros(train_df.shape[0])
    test_preds = np.zeros(test_df.shape[0])

    for train_index, test_index in skf.split(X, labels):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        X_train = X_train.reshape((-1,1))
        X_test = X_test.reshape((-1,1))

        model = IsolationForest(random_state=0).fit(X_train)
        preds = model.predict(X_test).clip(0,1).reshape(y_test.shape)
        validation_score = f1_score(y_test, preds)

        train_preds += model.predict(X_train_df.reshape(-1,1)).reshape(X_train_df.shape).clip(0,1)
        test_preds += model.predict(X_test_df.reshape(-1,1)).reshape(X_test_df.shape).clip(0,1)

    #     print('Validation score:' , validation_score)

        validation_scores.append(validation_score)
        models.append(model)
        
    train_df[f'{col}_anomaly'] = np.where(train_preds > 2, 1, 0)
    test_df[f'{col}_anomaly'] = np.where(test_preds > 2, 1, 0)
    return validation_scores

In [30]:
cols = ['fare','additional_fare','duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup']
for col in tqdm(cols):
    validation_scores = anomaly_pred(col)


100%|██████████| 6/6 [00:14<00:00,  2.33s/it]


In [31]:
def anomaly_pred_multi(cols, train_df=train_df, test_df=test_df, folds=3):
    labels = train_df['label'].values
    X = train_df[cols].values

    X_train_df = train_df[cols].values
    X_test_df = test_df[cols].values
    
    skf = StratifiedKFold(n_splits=3)

    validation_scores = []
    models = []

    train_preds = np.zeros(train_df.shape[0])
    test_preds = np.zeros(test_df.shape[0])
#     print(X.shape)

    for train_index, test_index in skf.split(X, labels):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        model = IsolationForest(random_state=0).fit(X_train)
        preds = model.predict(X_test).clip(0,1)
        validation_score = f1_score(y_test, preds)

        train_preds += model.predict(X_train_df).clip(0,1)
        test_preds += model.predict(X_test_df).clip(0,1)

    #     print('Validation score:' , validation_score)

        validation_scores.append(validation_score)
        models.append(model)
    name = '_'.join(cols)
    train_df[f'{name}_anomaly'] = np.where(train_preds > 2, 1, 0)
    test_df[f'{name}_anomaly'] = np.where(test_preds > 2, 1, 0)
    return validation_scores

In [32]:
cols = ['fare','additional_fare','duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup']
for i, col_1 in enumerate(cols):
    for col_2 in cols[i+1:]:
        validation_scores = anomaly_pred_multi([col_1,col_2])

In [33]:
for i, col_1 in enumerate(cols):
    for col_2 in cols[i+1:]:
        j = cols.index(col_2)
        for col_3 in cols[j+1:]:
            validation_scores = anomaly_pred_multi([col_1,col_2,col_3])

In [34]:
for i, col_1 in enumerate(cols):
    for col_2 in cols[i+1:]:
        j = cols.index(col_2)
        for col_3 in cols[j+1:]:
            k = cols.index(col_3)
            for col_4 in cols[k+1:]:                
                validation_scores = anomaly_pred_multi([col_1,col_2,col_3,col_4])
    

In [35]:
params = {
    'loss_function':'Logloss',
    'random_state':0,
    'early_stopping_rounds':50,
    'eval_metric':'F1',
#     'class_weights':class_weights
}

In [36]:
features = [
    'fare_anomaly',
    'additional_fare_anomaly', 
    'duration_anomaly',
    'meter_waiting_anomaly', 
    'meter_waiting_fare_anomaly',
    'meter_waiting_till_pickup_anomaly', 
    'additional_fare_duration_anomaly',
    'additional_fare_meter_waiting_anomaly',
    'additional_fare_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_anomaly', 
    'duration_meter_waiting_fare_anomaly',
    'duration_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_anomaly',
    'meter_waiting_meter_waiting_till_pickup_anomaly',
    'meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_anomaly',
    'additional_fare_duration_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_anomaly',
    'duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    
]

cat_features = [
    'additional_fare_anomaly', 
    'duration_anomaly',
    'meter_waiting_anomaly', 
    'meter_waiting_fare_anomaly',
    'meter_waiting_till_pickup_anomaly', 
    'additional_fare_duration_anomaly',
    'additional_fare_meter_waiting_anomaly',
    'additional_fare_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_anomaly', 
    'duration_meter_waiting_fare_anomaly',
    'duration_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_anomaly',
    'meter_waiting_meter_waiting_till_pickup_anomaly',
    'meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_anomaly',
    'additional_fare_duration_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_anomaly',
    'duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',

]

In [37]:
labels = train_df['label'].values
train_df = train_df.drop(['label'], axis=1)[features]

In [38]:
submission_pool = Pool(data=test_df[features], cat_features=cat_features)
train_df_pool = Pool(data=train_df[features], cat_features=cat_features)

In [39]:
skf = StratifiedKFold(n_splits=3)
validation_scores = []
submission_preds = np.zeros(submission_df.shape[0])
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
train_pools = []
models = []
for train_index, test_index in skf.split(train_df, labels):
    X_train, X_test = train_df.iloc[train_index,:], train_df.iloc[test_index,:]
    y_train, y_test = labels[train_index], labels[test_index]
    train_pool = Pool(data=X_train, label=y_train,cat_features=cat_features)
    test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)    
    model = CatBoostClassifier(**params)
    model.fit(X=train_pool, eval_set=test_pool,verbose=10)
    pred = model.predict(test_pool)
    validation_score = model.best_score_['validation']['F1']
    print('Validation f1',validation_score)
    validation_scores.append(validation_score)
    models.append(model)
    train_pools.append(train_pool)
    submission_preds += model.predict(submission_pool)
    train_preds += model.predict_proba(train_df_pool)[:,1]
    test_preds += model.predict_proba(submission_pool)[:,1]

Learning rate set to 0.057693
0:	learn: 0.9531844	test: 0.9530251	best: 0.9530251 (0)	total: 57.3ms	remaining: 57.3s
10:	learn: 0.9534970	test: 0.9531481	best: 0.9533160 (1)	total: 133ms	remaining: 11.9s
20:	learn: 0.9536295	test: 0.9533160	best: 0.9533247 (11)	total: 192ms	remaining: 8.93s
30:	learn: 0.9540672	test: 0.9534754	best: 0.9534754 (28)	total: 264ms	remaining: 8.26s
40:	learn: 0.9551255	test: 0.9549800	best: 0.9549800 (40)	total: 338ms	remaining: 7.9s
50:	learn: 0.9556092	test: 0.9552294	best: 0.9552294 (50)	total: 403ms	remaining: 7.49s
60:	learn: 0.9556051	test: 0.9552211	best: 0.9552294 (50)	total: 475ms	remaining: 7.3s
70:	learn: 0.9558632	test: 0.9552932	best: 0.9552932 (70)	total: 548ms	remaining: 7.17s
80:	learn: 0.9562671	test: 0.9554709	best: 0.9555597 (78)	total: 614ms	remaining: 6.97s
90:	learn: 0.9564894	test: 0.9555597	best: 0.9555597 (78)	total: 681ms	remaining: 6.8s
100:	learn: 0.9566067	test: 0.9555514	best: 0.9556485 (91)	total: 760ms	remaining: 6.77s
110:	l

In [40]:
train_df.to_csv('train_df_anomaly.csv',index=False)
test_df.to_csv('test_df_anomaly.csv',index=False)

## Shortened noise_pre_eda notebook

In [41]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')

data = train_df[train_df['label'] == 1].dropna()

In [42]:
cols = ['duration','meter_waiting','meter_waiting_fare','is_more_than_one_day']
X = data[cols].values
y = data['fare'].values

X_train_df = train_df[cols].values
X_test_df = test_df[cols].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_df = scaler.transform(X_train_df)
X_test_df = scaler.transform(X_test_df)

In [43]:
folds = 3
validation_scores = []
models = []

train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
kf = KFold(n_splits=folds)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    
#     model = CatBoostRegressor(**params)
#     model.fit(X=X_train,y=y_train,eval_set=(X_test,y_test))

    model = LinearRegression()
#     model = SVR()
    model.fit(X_train,y_train)
    
    pred = model.predict(X_test)
    score = mean_squared_error(y_test,pred) ** 0.5
    validation_scores.append(score)
    models.append(model)
    print('RMSE:', score)
    
    train_preds += model.predict(X_train_df)
    test_preds += model.predict(X_test_df)
    
train_preds /= folds
test_preds /= folds

RMSE: 96.5273471319087
RMSE: 82.29682215032592
RMSE: 82.63358797352826


In [44]:
train_df['predicted_fare'] = train_preds
test_df['predicted_fare'] = test_preds

train_df['predicted_fare_diff'] = train_df['fare'] - train_df['predicted_fare']
test_df['predicted_fare_diff'] = test_df['fare'] - test_df['predicted_fare']    

train_df['predicted_fare_diff_per_fare'] = train_df['predicted_fare_diff'] / (train_df['fare']+1)
test_df['predicted_fare_diff_per_fare'] = test_df['predicted_fare_diff'] / (test_df['fare']+1)

train_df['predicted_fare_diff_per_predicted_fare'] = train_df['predicted_fare_diff'] / (train_df['predicted_fare']+1)
test_df['predicted_fare_diff_per_predicted_fare'] = test_df['predicted_fare_diff'] / (test_df['predicted_fare']+1)

train_df['fare_per_distance'] = train_df['fare'] / (train_df['distance_km']+1)
test_df['fare_per_distance'] = test_df['fare'] / (test_df['distance_km']+1)

train_df['predicted_fare_per_distance'] = train_df['predicted_fare'] / (train_df['distance_km']+1)
test_df['predicted_fare_per_distance'] = test_df['predicted_fare'] / (test_df['distance_km']+1)

train_df['predicted_fare_diff_per_distance'] = train_df['predicted_fare_diff'] / (train_df['distance_km']+1)
test_df['predicted_fare_diff_per_distance'] = test_df['predicted_fare_diff'] / (test_df['distance_km']+1)

train_df['predicted_fare_diff_per_fare'] = train_df['predicted_fare_diff'] / (train_df['fare']+1)
test_df['predicted_fare_diff_per_fare'] = test_df['predicted_fare_diff'] / (test_df['fare']+1)

In [45]:
cols = ['meter_waiting','meter_waiting_fare','fare','is_more_than_one_day','cal_time_difference']

X = data[cols].values
y = data['duration'].values

X_train_df = train_df[cols].values
X_test_df = test_df[cols].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_df = scaler.transform(X_train_df)
X_test_df = scaler.transform(X_test_df)

In [46]:
folds = 3

validation_scores = []
models = []

train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
kf = KFold(n_splits=folds)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
   
    model = LinearRegression()
    model.fit(X_train,y_train)
    
    pred = model.predict(X_test)
    score = mean_squared_error(y_test,pred) ** 0.5
    validation_scores.append(score)
    models.append(model)
    print('RMSE:', score)
    
    train_preds += model.predict(X_train_df)
    test_preds += model.predict(X_test_df)
    
train_preds /= folds
test_preds /= folds

RMSE: 412.05683862976997
RMSE: 332.70147883644177
RMSE: 343.7877005393285


In [47]:
train_df['predicted_duration'] = train_preds
test_df['predicted_duration'] = test_preds

train_df['predicted_duration_diff'] = train_df['duration'] - train_df['predicted_duration']
test_df['predicted_duration_diff'] = test_df['duration'] - test_df['predicted_duration']    

train_df['predicted_duraton_diff_per_duraton'] = train_df['predicted_duration_diff'] / (train_df['duration']+1)
test_df['predicted_duraton_diff_per_duraton'] = test_df['predicted_duration_diff'] / (test_df['duration']+1)

train_df['predicted_duraton_diff_per_predicted_duration'] = train_df['predicted_duration_diff'] / (train_df['predicted_duration']+1)
test_df['predicted_duraton_diff_per_predicted_duration'] = test_df['predicted_duration_diff'] / (test_df['predicted_duration']+1)

train_df['predicted_duraton_diff_per_distance'] = train_df['predicted_duration_diff'] / (train_df['distance_km']+1)
test_df['predicted_duraton_diff_per_distance'] = test_df['predicted_duration_diff'] / (test_df['distance_km']+1)

train_df['fare_per_duration'] = train_df['fare'] / (train_df['duration']+1)
test_df['fare_per_duration'] = test_df['fare'] / (test_df['duration']+1)

train_df['predicted_fare_per_duration'] = train_df['predicted_fare'] / (train_df['predicted_duration']+1)
test_df['predicted_fare_per_duration'] = test_df['predicted_fare'] / (test_df['predicted_duration']+1)

train_df['predicted_fare_per_duration_diff'] = train_df['fare_per_duration'] - train_df['predicted_fare_per_duration']
test_df['predicted_fare_per_duration_diff'] = test_df['fare_per_duration'] - test_df['predicted_fare_per_duration']

train_df['avg_speed'] = train_df['distance_km'] / (train_df['duration'] + 1)
test_df['avg_speed'] = test_df['distance_km'] / (test_df['duration'] + 1)

train_df['predicted_avg_speed'] = train_df['distance_km'] / (train_df['predicted_duration'] + 1)
test_df['predicted_avg_speed'] = test_df['distance_km'] / (test_df['predicted_duration'] + 1)

train_df['predicted_avg_speed_diff'] = train_df['avg_speed'] - train_df['predicted_avg_speed']
test_df['predicted_avg_speed_diff'] = test_df['avg_speed'] - test_df['predicted_avg_speed']    

In [48]:
cols = ['duration','meter_waiting_fare','fare','cal_time_difference']

X = data[cols].values
y = data['meter_waiting'].values

X_train_df = train_df[cols].values
X_test_df = test_df[cols].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_df = scaler.transform(X_train_df)
X_test_df = scaler.transform(X_test_df)

In [49]:
folds = 3

validation_scores = []
models = []

train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
kf = KFold(n_splits=folds)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
        
    model = LinearRegression()
    model.fit(X_train,y_train)
    
    pred = model.predict(X_test)
    score = mean_squared_error(y_test,pred) ** 0.5
    validation_scores.append(score)
    models.append(model)
    print('RMSE:', score)
    
    train_preds += model.predict(X_train_df)
    test_preds += model.predict(X_test_df)
    
train_preds /= folds
test_preds /= folds

RMSE: 121.23549829752587
RMSE: 126.15607547905648
RMSE: 119.87616226350285


In [50]:
train_df['predicted_meter_waiting'] = train_preds
test_df['predicted_meter_waiting'] = test_preds

train_df['predicted_meter_waiting_diff'] = train_df['meter_waiting'] - train_df['predicted_meter_waiting']
test_df['predicted_meter_waiting_diff'] = test_df['meter_waiting'] - test_df['predicted_meter_waiting']

train_df['predicted_meter_waiting_diff_per_meter_waiting'] = train_df['predicted_meter_waiting_diff'] / (train_df['meter_waiting'] + 1)
test_df['predicted_meter_waiting_diff_per_meter_waiting'] = test_df['predicted_meter_waiting_diff'] / (test_df['meter_waiting'] + 1)

train_df['predicted_meter_waiting_diff_per_distance'] = train_df['predicted_meter_waiting_diff'] / (train_df['distance_km'] + 1)
test_df['predicted_meter_waiting_diff_per_distance'] = test_df['predicted_meter_waiting_diff'] / (test_df['distance_km'] + 1)

train_df['predicted_meter_waiting_diff_per_predicted_meter_waiting'] = train_df['predicted_meter_waiting_diff'] / (train_df['predicted_meter_waiting'] + 1)
test_df['predicted_meter_waiting_diff_per_predicted_meter_waiting'] = test_df['predicted_meter_waiting_diff'] / (test_df['predicted_meter_waiting'] + 1)

train_df['meter_waiting_per_duration'] = train_df['meter_waiting'] / (train_df['duration']+1)
test_df['meter_waiting_per_duration'] = test_df['meter_waiting'] / (test_df['duration']+1)

train_df['predicted_meter_waiting_per_duration'] = train_df['predicted_meter_waiting'] / (train_df['predicted_duration']+1)
test_df['predicted_meter_waiting_per_duration'] = test_df['predicted_meter_waiting'] / (test_df['predicted_duration']+1)

train_df['predicted_meter_waiting_per_duration_diff'] = train_df['meter_waiting_per_duration'] - train_df['predicted_meter_waiting_per_duration']
test_df['predicted_meter_waiting_per_duration_diff'] = test_df['meter_waiting_per_duration'] - test_df['predicted_meter_waiting_per_duration']

In [51]:
cols = ['duration','meter_waiting','fare','is_more_than_one_day','cal_time_difference']

X = data[cols].values
y = data['meter_waiting_fare'].values

X_train_df = train_df[cols].values
X_test_df = test_df[cols].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_df = scaler.transform(X_train_df)
X_test_df = scaler.transform(X_test_df)

In [52]:
folds = 3

validation_scores = []
models = []

train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
kf = KFold(n_splits=folds)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = LinearRegression()
    model.fit(X_train,y_train)
    
    pred = model.predict(X_test)
    score = mean_squared_error(y_test,pred) ** 0.5
    validation_scores.append(score)
    models.append(model)
    print('RMSE:', score)
    
    train_preds += model.predict(X_train_df)
    test_preds += model.predict(X_test_df)
    
train_preds /= folds
test_preds /= folds

RMSE: 7.21561205286957
RMSE: 7.758602065112114
RMSE: 8.8391720955996


In [53]:
train_df['predicted_meter_waiting_fare'] = train_preds
test_df['predicted_meter_waiting_fare'] = test_preds

train_df['predicted_meter_waiting_fare_diff'] = train_df['meter_waiting_fare'] - train_df['predicted_meter_waiting_fare']
test_df['predicted_meter_waiting_fare_diff'] = test_df['meter_waiting_fare'] - test_df['predicted_meter_waiting_fare']

train_df['predicted_meter_waiting_fare_diff_per_meter_waiting_fare'] = train_df['predicted_meter_waiting_fare_diff'] / (train_df['meter_waiting_fare']+1)
test_df['predicted_meter_waiting_fare_diff_per_meter_waiting_fare'] = test_df['predicted_meter_waiting_fare_diff'] / (test_df['meter_waiting_fare']+1)

train_df['predicted_meter_waiting_fare_diff_per_distance'] = train_df['predicted_meter_waiting_fare_diff'] / (train_df['distance_km']+1)
test_df['predicted_meter_waiting_fare_diff_per_distance'] = test_df['predicted_meter_waiting_fare_diff'] / (test_df['distance_km']+1)

train_df['predicted_meter_waiting_fare_diff_per_predicted_meter_waiting_fare'] = train_df['predicted_meter_waiting_fare_diff'] / (train_df['predicted_meter_waiting_fare']+1)
test_df['predicted_meter_waiting_fare_diff_per_predicted_meter_waiting_fare'] = test_df['predicted_meter_waiting_fare_diff'] / (test_df['predicted_meter_waiting_fare']+1)

train_df['meter_waiting_fare_per_meter_waiting'] = train_df['meter_waiting_fare'] / train_df['meter_waiting']
test_df['meter_waiting_fare_per_meter_waiting'] = test_df['meter_waiting_fare'] / test_df['meter_waiting']

train_df['predicted_meter_waiting_fare_per_meter_waiting'] = train_df['predicted_meter_waiting_fare'] / train_df['predicted_meter_waiting']
test_df['predicted_meter_waiting_fare_per_meter_waiting'] = test_df['predicted_meter_waiting_fare'] / test_df['predicted_meter_waiting']

train_df['predicted_meter_waiting_fare_per_meter_waiting_diff'] = train_df['meter_waiting_fare_per_meter_waiting'] - train_df['predicted_meter_waiting_fare_per_meter_waiting']
test_df['predicted_meter_waiting_fare_per_meter_waiting_diff'] = test_df['meter_waiting_fare_per_meter_waiting'] - test_df['predicted_meter_waiting_fare_per_meter_waiting']

train_df['meter_waiting_fare_per_duration'] = train_df['meter_waiting_fare'] / train_df['duration']
test_df['meter_waiting_fare_per_duration'] = test_df['meter_waiting_fare'] / test_df['duration']

train_df['predicted_meter_waiting_fare_per_duration'] = train_df['predicted_meter_waiting_fare'] / train_df['predicted_duration']
test_df['predicted_meter_waiting_fare_per_duration'] = test_df['predicted_meter_waiting_fare'] / test_df['predicted_duration']

train_df['predicted_meter_waiting_fare_per_duration_diff'] = train_df['meter_waiting_fare_per_duration'] - train_df['predicted_meter_waiting_fare_per_duration']
test_df['predicted_meter_waiting_fare_per_duration_diff'] = test_df['meter_waiting_fare_per_duration'] - test_df['predicted_meter_waiting_fare_per_duration']

In [54]:
cols = ['meter_waiting_fare_per_duration','meter_waiting_per_duration','fare_per_duration']
data = train_df[train_df['label'] == 1].dropna()

X = data[cols].values
y = data['additional_fare'].values

X_train_df = train_df[cols].values
X_test_df = test_df[cols].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_df = np.nan_to_num(scaler.transform(X_train_df))
X_test_df = np.nan_to_num(scaler.transform(X_test_df))

In [55]:
folds = 3

validation_scores = []
models = []

train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
kf = KFold(n_splits=folds)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

#     model = CatBoostRegressor(**params)
#     model.fit(X=X_train,y=y_train,eval_set=(X_test,y_test))
   
    model = LinearRegression()
    model.fit(X_train,y_train)
    
    pred = model.predict(X_test)
    score = mean_squared_error(y_test,pred) ** 0.5
    validation_scores.append(score)
    models.append(model)
    print('RMSE:', score)
    
    train_preds += model.predict(X_train_df)
    test_preds += model.predict(X_test_df)
    
train_preds /= folds
test_preds /= folds

RMSE: 8.720292350311684
RMSE: 11.875141951981368
RMSE: 12.088516557325503


In [56]:
train_df['predicted_additional_fare'] = train_preds
test_df['predicted_additional_fare'] = test_preds

train_df['predicted_additional_fare_diff'] = train_df['additional_fare'] - train_df['predicted_additional_fare']
test_df['predicted_additional_fare_diff'] = test_df['additional_fare'] - test_df['predicted_additional_fare']

train_df['predicted_additional_fare_diff_per_additional_fare'] = train_df['predicted_additional_fare_diff'] / (train_df['additional_fare']+1)
test_df['predicted_additional_fare_diff_per_additional_fare'] = test_df['predicted_additional_fare_diff'] / (test_df['additional_fare']+1)

train_df['predicted_addtional_fare_per_fare'] = train_df['predicted_additional_fare'] / (train_df['predicted_fare']+1)
test_df['predicted_addtional_fare_per_fare'] = test_df['predicted_additional_fare'] / (test_df['predicted_fare']+1)

train_df['addtional_fare_per_fare'] = train_df['additional_fare'] / (train_df['fare']+1)
test_df['addtional_fare_per_fare'] = test_df['additional_fare'] / (test_df['fare']+1)

train_df['addtional_fare_per_distance'] = train_df['additional_fare'] / (train_df['distance_km']+1)
test_df['addtional_fare_per_distance'] = test_df['additional_fare'] / (test_df['distance_km']+1)

train_df['predicted_addtional_fare_per_distance'] = train_df['predicted_additional_fare'] / (train_df['distance_km']+1)
test_df['predicted_addtional_fare_per_distance'] = test_df['predicted_additional_fare'] / (test_df['distance_km']+1)

train_df['predicted_addtional_fare_diff_per_distance'] = train_df['predicted_additional_fare_diff'] / (train_df['distance_km']+1)
test_df['predicted_addtional_fare_diff_per_distance'] = test_df['predicted_additional_fare_diff'] / (test_df['distance_km']+1)

train_df['addtional_fare_per_duration'] = train_df['additional_fare'] / (train_df['duration']+1)
test_df['addtional_fare_per_duration'] = test_df['additional_fare'] / (test_df['duration']+1)

train_df['predicted_addtional_fare_per_duration'] = train_df['predicted_additional_fare'] / (train_df['predicted_duration']+1)
test_df['predicted_addtional_fare_per_duration'] = test_df['predicted_additional_fare'] / (test_df['predicted_duration']+1)

train_df['fare-additional_fare'] = train_df['fare'] - train_df['additional_fare']
test_df['fare-additional_fare'] = test_df['fare'] - test_df['additional_fare']

train_df['predicted_fare-additional_fare'] = train_df['predicted_fare'] - train_df['predicted_additional_fare']
test_df['predicted_fare-additional_fare'] = test_df['predicted_fare'] - test_df['predicted_additional_fare']

train_df['fare-additional_fare-meter_waiting_fare'] = train_df['fare'] - (train_df['additional_fare'] + train_df['meter_waiting_fare'])
test_df['fare-additional_fare-meter_waiting_fare'] = test_df['fare'] - (test_df['additional_fare'] + test_df['meter_waiting_fare'])

train_df['predicted_fare-additional_fare-meter_waiting_fare'] = train_df['predicted_fare'] - (train_df['predicted_additional_fare'] + train_df['predicted_meter_waiting_fare'])
test_df['predicted_fare-additional_fare-meter_waiting_fare'] = test_df['predicted_fare'] - (test_df['predicted_additional_fare'] + test_df['predicted_meter_waiting_fare'])

train_df['fare-additional_fare_per_distance'] = train_df['fare-additional_fare'] / (train_df['distance_km']+1)
test_df['fare-additional_fare_per_distance'] = test_df['fare-additional_fare'] / (test_df['distance_km']+1)

train_df['predicted_fare-additional_fare_per_distance'] = train_df['predicted_fare-additional_fare'] / (train_df['distance_km']+1)
test_df['predicted_fare-additional_fare_per_distance'] = test_df['predicted_fare-additional_fare'] / (test_df['distance_km']+1)

train_df['fare-additional_fare_per_duration'] = train_df['fare-additional_fare'] / (train_df['duration']+1)
test_df['fare-additional_fare_per_duration'] = test_df['fare-additional_fare'] / (test_df['duration']+1)

train_df['predicted_fare-additional_fare_per_duration'] = train_df['predicted_fare-additional_fare'] / (train_df['predicted_duration']+1)
test_df['predicted_fare-additional_fare_per_duration'] = test_df['predicted_fare-additional_fare'] / (test_df['predicted_duration']+1)

train_df['fare-additional_fare-meter_waiting_fare_per_distance'] = train_df['fare-additional_fare-meter_waiting_fare'] / (train_df['distance_km']+1)
test_df['fare-additional_fare-meter_waiting_fare_per_distance'] = test_df['fare-additional_fare-meter_waiting_fare'] / (test_df['distance_km']+1)

train_df['predicted_fare-additional_fare-meter_waiting_fare_per_distance'] = train_df['predicted_fare-additional_fare-meter_waiting_fare'] / (train_df['distance_km']+1)
test_df['predicted_fare-additional_fare-meter_waiting_fare_per_distance'] = test_df['predicted_fare-additional_fare-meter_waiting_fare'] / (test_df['distance_km']+1)

train_df['fare-additional_fare-meter_waiting_fare_per_duration'] = train_df['fare-additional_fare-meter_waiting_fare'] / (train_df['duration']+1)
test_df['fare-additional_fare-meter_waiting_fare_per_duration'] = test_df['fare-additional_fare-meter_waiting_fare'] / (test_df['duration']+1)

train_df['predicted_fare-additional_fare-meter_waiting_fare_per_duration'] = train_df['predicted_fare-additional_fare-meter_waiting_fare'] / (train_df['predicted_duration']+1)
test_df['predicted_fare-additional_fare-meter_waiting_fare_per_duration'] = test_df['predicted_fare-additional_fare-meter_waiting_fare'] / (test_df['predicted_duration']+1)

In [57]:
data = train_df[train_df['label'] == 1].dropna()
y = data['meter_waiting_till_pickup'].values
X = data.drop(['label','meter_waiting_till_pickup'],axis=1)
cols = X.columns
X = X.values

X_train_df = train_df[cols].values
X_test_df = test_df[cols].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_df = np.nan_to_num(scaler.transform(X_train_df))
X_test_df = np.nan_to_num(scaler.transform(X_test_df))

In [58]:
params = {
    'loss_function':'RMSE',
    'random_state':0,
    'early_stopping_rounds':50,
    'eval_metric':'RMSE'
}

In [59]:
folds = 3
validation_scores = []
models = []

train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
kf = KFold(n_splits=folds)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = CatBoostRegressor(**params)
    model.fit(X=X_train,y=y_train,eval_set=(X_test,y_test),verbose=10)
   

    pred = model.predict(X_test)
    score = mean_squared_error(y_test,pred) ** 0.5
    validation_scores.append(score)
    models.append(model)
    print('RMSE:', score)
    
    train_preds += model.predict(X_train_df)
    test_preds += model.predict(X_test_df)
    
train_preds /= folds
test_preds /= folds

Learning rate set to 0.071025
0:	learn: 118.6085647	test: 752.4014533	best: 752.4014533 (0)	total: 16.1ms	remaining: 16s
10:	learn: 115.6279704	test: 751.4615819	best: 751.4615819 (10)	total: 108ms	remaining: 9.68s
20:	learn: 114.1567627	test: 751.0719888	best: 751.0719888 (20)	total: 196ms	remaining: 9.13s
30:	learn: 113.1604569	test: 750.7494121	best: 750.7494121 (30)	total: 301ms	remaining: 9.4s
40:	learn: 112.3488130	test: 750.3135685	best: 750.3135685 (40)	total: 394ms	remaining: 9.22s
50:	learn: 111.6237013	test: 750.1796473	best: 750.1796473 (50)	total: 497ms	remaining: 9.25s
60:	learn: 110.9276689	test: 750.1443872	best: 750.1353974 (57)	total: 594ms	remaining: 9.15s
70:	learn: 110.3372737	test: 749.9718913	best: 749.9536518 (63)	total: 690ms	remaining: 9.03s
80:	learn: 109.6922453	test: 749.9560205	best: 749.9450179 (72)	total: 796ms	remaining: 9.04s
90:	learn: 109.0438206	test: 749.8348078	best: 749.8250639 (83)	total: 914ms	remaining: 9.13s
100:	learn: 108.4398849	test: 749.

In [60]:
train_df['predicted_meter_waiting_till_pickup'] = train_preds
test_df['predicted_meter_waiting_till_pickup'] = test_preds

train_df['predicted_meter_waiting_till_pickup_diff'] = train_df['meter_waiting_till_pickup'] - train_df['predicted_meter_waiting_till_pickup']
test_df['predicted_meter_waiting_till_pickup_diff'] = test_df['meter_waiting_till_pickup'] - test_df['predicted_meter_waiting_till_pickup']

train_df['predicted_meter_waiting_till_pickup_diff_per_meter_waiting_till_pickup'] = train_df['predicted_meter_waiting_till_pickup_diff'] / (train_df['meter_waiting_till_pickup']+1)
test_df['predicted_meter_waiting_till_pickup_diff_per_meter_waiting_till_pickup'] = test_df['predicted_meter_waiting_till_pickup_diff'] / (test_df['meter_waiting_till_pickup']+1)

train_df['meter_waiting_till_pickup_per_meter_waiting'] = train_df['meter_waiting_till_pickup'] / (train_df['meter_waiting'] + 1)
test_df['meter_waiting_till_pickup_per_meter_waiting'] = test_df['meter_waiting_till_pickup'] / (test_df['meter_waiting'] + 1)

train_df['predicted_meter_waiting_till_pickup_per_meter_waiting'] = train_df['predicted_meter_waiting_till_pickup'] / (train_df['predicted_meter_waiting'] + 1)
test_df['predicted_meter_waiting_till_pickup_per_meter_waiting'] = test_df['predicted_meter_waiting_till_pickup'] / (test_df['predicted_meter_waiting'] + 1)

train_df['predicted_meter_waiting_till_pickup_per_meter_waiting_diff'] = train_df['meter_waiting_till_pickup_per_meter_waiting'] - train_df['predicted_meter_waiting_till_pickup_per_meter_waiting']
test_df['predicted_meter_waiting_till_pickup_per_meter_waiting_diff'] = test_df['meter_waiting_till_pickup_per_meter_waiting'] - test_df['predicted_meter_waiting_till_pickup_per_meter_waiting']

train_df['meter_waiting_after_pickup'] = train_df['meter_waiting'] - train_df['meter_waiting_till_pickup']
test_df['meter_waiting_after_pickup'] = test_df['meter_waiting'] - test_df['meter_waiting_till_pickup']

train_df['predicted_meter_waiting_after_pickup'] = train_df['predicted_meter_waiting'] - train_df['predicted_meter_waiting_till_pickup']
test_df['predicted_meter_waiting_after_pickup'] = test_df['predicted_meter_waiting'] - test_df['predicted_meter_waiting_till_pickup']

train_df['meter_waiting_after_pickup_per_duration'] = train_df['meter_waiting_after_pickup'] / (train_df['duration'] + 1)
test_df['meter_waiting_after_pickup_per_duration'] = test_df['meter_waiting_after_pickup'] / (test_df['duration'] + 1)

train_df['predicted_meter_waiting_after_pickup_per_duration'] = train_df['predicted_meter_waiting_after_pickup'] / (train_df['predicted_duration'] + 1)
test_df['predicted_meter_waiting_after_pickup_per_duration'] = test_df['predicted_meter_waiting_after_pickup'] / (test_df['predicted_duration'] + 1)

train_df['meter_waiting_till_pickup_per_duration'] = train_df['meter_waiting_till_pickup'] / (train_df['duration'] + 1)
test_df['meter_waiting_till_pickup_per_duration'] = test_df['meter_waiting_till_pickup'] / (test_df['duration'] + 1)

train_df['predicted_meter_waiting_till_pickup_per_duration'] = train_df['predicted_meter_waiting_till_pickup'] / (train_df['predicted_duration'] + 1)
test_df['predicted_meter_waiting_till_pickup_per_duration'] = test_df['predicted_meter_waiting_till_pickup'] / (test_df['predicted_duration'] + 1)

train_df['meter_waiting_till_pickup_per_distance'] = train_df['meter_waiting_till_pickup'] / (train_df['distance_km'] + 1)
test_df['meter_waiting_till_pickup_per_distance'] = test_df['meter_waiting_till_pickup'] / (test_df['distance_km'] + 1)

train_df['predicted_meter_waiting_till_pickup_per_distance'] = train_df['predicted_meter_waiting_till_pickup'] / (train_df['distance_km'] + 1)
test_df['predicted_meter_waiting_till_pickup_per_distance'] = test_df['predicted_meter_waiting_till_pickup'] / (test_df['distance_km'] + 1)

train_df['meter_waiting_after_pickup_per_distance'] = train_df['meter_waiting_after_pickup'] / (train_df['distance_km'] + 1)
test_df['meter_waiting_after_pickup_per_distance'] = test_df['meter_waiting_after_pickup'] / (test_df['distance_km'] + 1)

train_df['predicted_meter_waiting_after_pickup_per_distance'] = train_df['predicted_meter_waiting_after_pickup'] / (train_df['distance_km'] + 1)
test_df['predicted_meter_waiting_after_pickup_per_distance'] = test_df['predicted_meter_waiting_after_pickup'] / (test_df['distance_km'] + 1)

train_df['meter_waiting_till_pickup_per_fare'] = train_df['meter_waiting_till_pickup'] / (train_df['fare'] + 1)
test_df['meter_waiting_till_pickup_per_fare'] = test_df['meter_waiting_till_pickup'] / (test_df['fare'] + 1)

train_df['predicted_meter_waiting_till_pickup_per_fare'] = train_df['predicted_meter_waiting_till_pickup'] / (train_df['predicted_fare'] + 1)
test_df['predicted_meter_waiting_till_pickup_per_fare'] = test_df['predicted_meter_waiting_till_pickup'] / (test_df['predicted_fare'] + 1)

train_df['meter_waiting_after_pickup_per_fare'] = train_df['meter_waiting_after_pickup'] / (train_df['fare'] + 1)
test_df['meter_waiting_after_pickup_per_fare'] = test_df['meter_waiting_after_pickup'] / (test_df['fare'] + 1)

train_df['predicted_meter_waiting_after_pickup_per_fare'] = train_df['predicted_meter_waiting_after_pickup'] / (train_df['predicted_fare'] + 1)
test_df['predicted_meter_waiting_after_pickup_per_fare'] = test_df['predicted_meter_waiting_after_pickup'] / (test_df['predicted_fare'] + 1)

train_df['meter_waiting_till_pickup_per_meter_waiting_fare'] = train_df['meter_waiting_till_pickup'] / (train_df['meter_waiting_fare'] + 1)
test_df['meter_waiting_till_pickup_per_meter_waiting_fare'] = test_df['meter_waiting_till_pickup'] / (test_df['meter_waiting_fare'] + 1)

train_df['predicted_meter_waiting_till_pickup_per_meter_waiting_fare'] = train_df['predicted_meter_waiting_till_pickup'] / (train_df['predicted_meter_waiting_fare'] + 1)
test_df['predicted_meter_waiting_till_pickup_per_meter_waiting_fare'] = test_df['predicted_meter_waiting_till_pickup'] / (test_df['predicted_meter_waiting_fare'] + 1)

train_df['meter_waiting_after_pickup_per_meter_waiting_fare'] = train_df['meter_waiting_after_pickup'] / (train_df['meter_waiting_fare'] + 1)
test_df['meter_waiting_after_pickup_per_meter_waiting_fare'] = test_df['meter_waiting_after_pickup'] / (test_df['meter_waiting_fare'] + 1)

train_df['predicted_meter_waiting_after_pickup_per_meter_waiting_fare'] = train_df['predicted_meter_waiting_after_pickup'] / (train_df['predicted_meter_waiting_fare'] + 1)
test_df['predicted_meter_waiting_after_pickup_per_meter_waiting_fare'] = test_df['predicted_meter_waiting_after_pickup'] / (test_df['predicted_meter_waiting_fare'] + 1)

In [61]:
train_anomaly = pd.read_csv('train_df_anomaly.csv')
test_anomaly = pd.read_csv('test_df_anomaly.csv')

In [62]:
anomaly_columns = [
    'fare_anomaly',
    'additional_fare_anomaly', 
    'duration_anomaly',
    'meter_waiting_anomaly', 
    'meter_waiting_fare_anomaly',
    'meter_waiting_till_pickup_anomaly', 
    'additional_fare_duration_anomaly',
    'additional_fare_meter_waiting_anomaly',
    'additional_fare_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_anomaly', 
    'duration_meter_waiting_fare_anomaly',
    'duration_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_anomaly',
    'meter_waiting_meter_waiting_till_pickup_anomaly',
    'meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_anomaly',
    'additional_fare_duration_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_anomaly',
    'duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
]

In [63]:
for col in anomaly_columns:
    train_df[col] = 1-train_anomaly[col]
    test_df[col] = 1-test_anomaly[col]

In [64]:
anomaly_multiplicatives = {
    'fare_anomaly':[
        'predicted_fare_diff',
        'predicted_fare_diff_per_fare',
        'predicted_fare_diff_per_distance',
    ],
    'additional_fare_anomaly':[
        'predicted_additional_fare_diff',
        'predicted_additional_fare_diff_per_additional_fare',
        'predicted_addtional_fare_per_distance',
    ],
    'duration_anomaly':[
        'predicted_duration_diff', 
        'predicted_duraton_diff_per_duraton',
        'predicted_duraton_diff_per_distance', 
    ],
    'meter_waiting_anomaly':[
        'predicted_meter_waiting_diff',
        'predicted_meter_waiting_diff_per_meter_waiting',
        'predicted_meter_waiting_diff_per_distance'
    ],
    'meter_waiting_fare_anomaly':[
        'predicted_meter_waiting_fare_diff',
        'predicted_meter_waiting_fare_diff_per_meter_waiting_fare',
        'predicted_meter_waiting_fare_diff_per_distance'
    ]
    
}

In [65]:
new_cols = []
for col1 in anomaly_multiplicatives:
    for col2 in anomaly_multiplicatives[col1]:
        name = f'{col1}_{col2}_prod'
        train_df[name] = train_df[col1] * train_df[col2]
        test_df[name] = test_df[col1] * test_df[col2]
        new_cols.append(name)

In [66]:
train_df['predicted_duraton_diff_per_duraton@predicted_duraton_diff_per_distance'] = train_df['predicted_duraton_diff_per_duraton'] * train_df['predicted_duraton_diff_per_distance']
test_df['predicted_duraton_diff_per_duraton@predicted_duraton_diff_per_distance'] = test_df['predicted_duraton_diff_per_duraton'] * test_df['predicted_duraton_diff_per_distance']

In [67]:
def normalize_diff(col_name):
    normalizer = StandardScaler()
    normalizer.fit(train_df[train_df['label'] == 1][col_name].values.reshape(-1,1))

    train_df[f'{col_name}_normalized'] = normalizer.transform(train_df[col_name].values.reshape(-1,1))
    test_df[f'{col_name}_normalized'] = normalizer.transform(test_df[col_name].values.reshape(-1,1))

In [68]:
diff_cols = [
    'predicted_fare_diff',
    'predicted_fare_diff_per_fare',
    'predicted_fare_diff_per_predicted_fare', 
    'predicted_fare_diff_per_distance',
    'predicted_duraton_diff_per_duraton',
    'predicted_duraton_diff_per_predicted_duration', 
    'predicted_fare_per_duration_diff',
    'predicted_avg_speed_diff',
    'predicted_meter_waiting_diff',
    'predicted_meter_waiting_diff_per_meter_waiting',
    'predicted_meter_waiting_diff_per_predicted_meter_waiting',
    'predicted_meter_waiting_per_duration_diff',
    'predicted_meter_waiting_fare_diff',
    'predicted_meter_waiting_fare_diff_per_meter_waiting_fare',
    'predicted_meter_waiting_fare_diff_per_predicted_meter_waiting_fare',
    'predicted_meter_waiting_fare_per_meter_waiting_diff',
    'predicted_meter_waiting_fare_per_duration_diff',
    'predicted_additional_fare_diff',
    'predicted_additional_fare_diff_per_additional_fare'
]
for col in diff_cols:
    normalize_diff(col)

In [69]:
def mean_value(col):
    grouping_order = ['pick_cluster','pickup_timeslot']
    group = train_df[train_df['label'] == 1].groupby(grouping_order)[col].mean()
    def f(row):
        return group[row['pick_cluster']][row['pickup_timeslot']]
    return f

In [70]:
def mean_encoding(col):
    train_df[f'{col}_mean'] = train_df.apply(mean_value(col),axis=1)
    test_df[f'{col}_mean'] = test_df.apply(mean_value(col),axis=1)
    
    train_df[f'{col}_mean_diff'] = train_df[f'{col}_mean'] - train_df[col]
    test_df[f'{col}_mean_diff'] = test_df[f'{col}_mean'] - test_df[col]

In [71]:
mean_cols = [
    'fare_per_distance',
    'avg_speed', 
    'meter_waiting_per_duration', 
    'meter_waiting_fare_per_meter_waiting',
    'meter_waiting_fare_per_duration',
    'addtional_fare_per_fare',
    'addtional_fare_per_distance', 
    'addtional_fare_per_duration'
]
for col in mean_cols:
    mean_encoding(col)

In [72]:
cols = [
    'predicted_fare_diff',
    'predicted_duration_diff',
    'predicted_meter_waiting_diff',
    'predicted_meter_waiting_fare_diff',
    'predicted_additional_fare_diff',    
]

In [73]:
def col_bucket(column):
    std = train_df[train_df['label']==1][column].std()
    name = f'{column}_bucket'
    train_df[name] = np.round((train_df[column]/std)+1).astype(int)
    test_df[name] = np.round((test_df[column]/std)+1).astype(int)    

In [74]:
for each in cols:
    col_bucket(each)

In [75]:
bin_multiplicatives = {
    'predicted_fare_diff_bucket':[
        'fare',
        'predicted_fare',
        'fare_per_distance',
        'predicted_fare_per_distance',         
    ],
    'predicted_duration_diff_bucket':[
        'duration',
        'predicted_duration',
        'avg_speed', 
        'predicted_avg_speed',         
    ],
    'predicted_meter_waiting_diff_bucket':[
        'meter_waiting', 
        'predicted_meter_waiting', 
        'meter_waiting_per_duration', 
        'predicted_meter_waiting_per_duration',
    ],
    'predicted_meter_waiting_fare_diff_bucket':[
        'meter_waiting_fare',
        'predicted_meter_waiting_fare',
        'meter_waiting_fare_per_meter_waiting',
        'predicted_meter_waiting_fare_per_meter_waiting',
        'meter_waiting_fare_per_duration',
        'predicted_meter_waiting_fare_per_duration',
    ],
    'predicted_additional_fare_diff':[
        'additional_fare',
        'predicted_additional_fare', 
        'predicted_addtional_fare_per_fare', 
        'addtional_fare_per_fare',
        'addtional_fare_per_distance', 
        'predicted_addtional_fare_per_distance',
        'addtional_fare_per_duration', 
        'predicted_addtional_fare_per_duration',
    ]
    
}

In [76]:
for bucket in tqdm(bin_multiplicatives):
    for col in bin_multiplicatives[bucket]:
        name = f'{bucket}@{col}'
        train_df[name] = train_df[bucket] * train_df[col]
        test_df[name] = test_df[bucket] * test_df[col]

100%|██████████| 5/5 [00:00<00:00, 102.61it/s]


In [77]:
train_df.to_csv('train_df_final.csv',index=False)
test_df.to_csv('test_df_final.csv',index=False)

# Model 1

In [80]:
train_df = pd.read_csv('train_df_final.csv')
train_df = train_df.fillna(0)
test_df = pd.read_csv('test_df_final.csv')
test_df = test_df.fillna(0)
submission_df = pd.read_csv('sample_submission.csv')

In [82]:
y = train_df['label'].values

In [83]:
original_cat_cols = [
    'pickup_hour',
    'drop_hour',
#     'pick_cluster'
]

In [84]:
features = [
    'fare_per_distance',
    'fare_per_duration',
    'avg_speed',
    'meter_waiting_per_duration',
    'meter_waiting_fare_per_meter_waiting',
    'meter_waiting_fare_per_duration',
    'addtional_fare_per_fare',
    'addtional_fare_per_distance',
    'addtional_fare_per_duration',
    'fare-additional_fare_per_distance',
    'fare-additional_fare_per_duration',
    'fare-additional_fare-meter_waiting_fare_per_distance',
    'fare-additional_fare-meter_waiting_fare_per_duration',
    'meter_waiting_till_pickup_per_meter_waiting',
    'meter_waiting_after_pickup_per_duration',
    'meter_waiting_till_pickup_per_duration',
    'meter_waiting_till_pickup_per_distance',
    'meter_waiting_after_pickup_per_distance',
    'meter_waiting_till_pickup_per_fare',
    'meter_waiting_after_pickup_per_fare',
    'meter_waiting_till_pickup_per_meter_waiting_fare',
    'meter_waiting_after_pickup_per_meter_waiting_fare',    
    'meter_waiting_till_pickup'
]

cat_features = []

In [85]:
catboost_params = {
    'loss_function':'Logloss',
    'random_state':0,
    'early_stopping_rounds':50,
    'eval_metric':'F1',
    'border_count':512
}

In [86]:
features += original_cat_cols
cat_features += original_cat_cols

In [88]:
train = train_df[features]
test = test_df[features]
y = train_df['label']

In [92]:
neumeric_cols_to_multiply = [
    'fare_per_distance',    
    'avg_speed',    
    'meter_waiting_fare_per_meter_waiting',
    'meter_waiting_till_pickup'
]

encoding_cols = []
for col1 in original_cat_cols:
    for col2 in neumeric_cols_to_multiply:
        name = f'{col1}@{col2}'
        train_df[name] = train_df[col1] * train_df[col2]
        test_df[name] = test_df[col1] * test_df[col2]
        encoding_cols.append(name)

In [93]:
features += encoding_cols

In [94]:
train = train_df[features]
test = test_df[features]
y = train_df['label']

In [95]:
submission_pool = Pool(data=test_df[features], cat_features=cat_features)

In [96]:
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
train_class = np.zeros(train_df.shape[0])
test_class = np.zeros(test_df.shape[0])
skf = StratifiedKFold(n_splits=3)
validation_scores = []
models = []
train_pools = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_pool = Pool(data=X_train, label=y_train,cat_features=cat_features)
    train_pools.append(train_pool)
    test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)    
    model = CatBoostClassifier(**catboost_params)
    model.fit(X=train_pool, eval_set=test_pool,verbose=10)
    train_preds[test_index] = model.predict_proba(test_pool)[:,1]
    train_class[test_index] = model.predict(test_pool)
    test_preds += model.predict_proba(submission_pool)[:,1]/3
    test_class += model.predict(submission_pool)
    validation_scores.append(f1_score(y_test,model.predict(test_pool),average='micro'))
    models.append(model)
test_class = np.where(test_class > 2, 1, 0)

Learning rate set to 0.057693
0:	learn: 0.9666778	test: 0.9632290	best: 0.9632290 (0)	total: 22.3ms	remaining: 22.3s
10:	learn: 0.9699035	test: 0.9739840	best: 0.9739840 (10)	total: 182ms	remaining: 16.4s
20:	learn: 0.9728780	test: 0.9763629	best: 0.9764695 (16)	total: 349ms	remaining: 16.3s
30:	learn: 0.9745396	test: 0.9771838	best: 0.9771838 (30)	total: 483ms	remaining: 15.1s
40:	learn: 0.9756005	test: 0.9772510	best: 0.9774422 (36)	total: 634ms	remaining: 14.8s
50:	learn: 0.9765245	test: 0.9776969	best: 0.9777097 (46)	total: 767ms	remaining: 14.3s
60:	learn: 0.9772239	test: 0.9777905	best: 0.9779904 (52)	total: 915ms	remaining: 14.1s
70:	learn: 0.9777799	test: 0.9776927	best: 0.9779904 (52)	total: 1.1s	remaining: 14.4s
80:	learn: 0.9787173	test: 0.9772967	best: 0.9779904 (52)	total: 1.28s	remaining: 14.5s
90:	learn: 0.9790859	test: 0.9774840	best: 0.9779904 (52)	total: 1.41s	remaining: 14.1s
100:	learn: 0.9793222	test: 0.9779567	best: 0.9779904 (52)	total: 1.56s	remaining: 13.9s
Sto

In [97]:
submission_df['prediction'] = test_class
submission_df.to_csv('submission.csv',index=False)

# Model 2

In [99]:
train_df = pd.read_csv('train_df_final.csv')
train_df = train_df.fillna(0)
test_df = pd.read_csv('test_df_final.csv')
test_df = test_df.fillna(0)
submission_df = pd.read_csv('sample_submission.csv')

y = train_df['label'].values

In [100]:
original_cat_cols = [
    'pickup_hour',
    'drop_hour',
#     'pick_cluster'
]

In [101]:
features = [
    'fare_per_distance',
    'fare_per_duration',
    'avg_speed',
    'meter_waiting_per_duration',
    'meter_waiting_fare_per_meter_waiting',
    'meter_waiting_fare_per_duration',
    'addtional_fare_per_fare',
    'addtional_fare_per_distance',
    'addtional_fare_per_duration',
    'fare-additional_fare_per_distance',
    'fare-additional_fare_per_duration',
    'fare-additional_fare-meter_waiting_fare_per_distance',
    'fare-additional_fare-meter_waiting_fare_per_duration',
    'meter_waiting_till_pickup_per_meter_waiting',
    'meter_waiting_after_pickup_per_duration',
    'meter_waiting_till_pickup_per_duration',
    'meter_waiting_till_pickup_per_distance',
    'meter_waiting_after_pickup_per_distance',
    'meter_waiting_till_pickup_per_fare',
    'meter_waiting_after_pickup_per_fare',
    'meter_waiting_till_pickup_per_meter_waiting_fare',
    'meter_waiting_after_pickup_per_meter_waiting_fare',    
    'meter_waiting_till_pickup',
    
    'additional_fare',
    'duration',
    'meter_waiting',
    'meter_waiting_fare',
    
    'predicted_duration_diff',
    'predicted_fare_diff_per_predicted_fare',
    'predicted_fare_diff_per_fare',
    'predicted_fare_diff_per_distance',
]

cat_features = []

In [102]:
catboost_params = {
    'loss_function':'Logloss',
    'random_state':0,
    'early_stopping_rounds':50,
    'eval_metric':'F1',
    'border_count':512
}

In [103]:
features += original_cat_cols
cat_features += original_cat_cols

In [104]:
neumeric_cols_to_multiply = [
    'fare_per_distance',  
#     'fare_per_duration',
#     'fare-additional_fare_per_duration',
    'avg_speed',    
    'meter_waiting_fare_per_meter_waiting',
    'meter_waiting_till_pickup'
]

encoding_cols = []
for col1 in original_cat_cols:
    for col2 in neumeric_cols_to_multiply:
        name = f'{col1}@{col2}'
        train_df[name] = train_df[col1] * train_df[col2]
        test_df[name] = test_df[col1] * test_df[col2]
        encoding_cols.append(name)

In [106]:
special_features = []

train_df['pickup_timeslot@distance'] = (train_df['pickup_timeslot']+1) * train_df['distance_km']
test_df['pickup_timeslot@distance'] = (test_df['pickup_timeslot']+1) * test_df['distance_km']
special_features.append('pickup_timeslot@distance')

In [107]:
features += encoding_cols
features += special_features

In [109]:
train = train_df[features]
test = test_df[features]
y = train_df['label']

In [110]:
submission_pool = Pool(data=test_df[features], cat_features=cat_features)


In [111]:
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
train_class = np.zeros(train_df.shape[0])
test_class = np.zeros(test_df.shape[0])
skf = StratifiedKFold(n_splits=3)
validation_scores = []
models = []
train_pools = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_pool = Pool(data=X_train, label=y_train,cat_features=cat_features)
    train_pools.append(train_pool)
    test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)    
    model = CatBoostClassifier(**catboost_params)
    model.fit(X=train_pool, eval_set=test_pool,verbose=10)
    train_preds[test_index] = model.predict_proba(test_pool)[:,1]
    train_class[test_index] = model.predict(test_pool)
    test_preds += model.predict_proba(submission_pool)[:,1]/3
    test_class += model.predict(submission_pool)
    validation_scores.append(f1_score(y_test,model.predict(test_pool),average='micro'))
    models.append(model)
test_class = np.where(test_class > 2, 1, 0)

Learning rate set to 0.057693
0:	learn: 0.9682270	test: 0.9657256	best: 0.9657256 (0)	total: 26.3ms	remaining: 26.3s
10:	learn: 0.9740705	test: 0.9766597	best: 0.9766597 (10)	total: 217ms	remaining: 19.5s
20:	learn: 0.9756424	test: 0.9774883	best: 0.9775905 (15)	total: 376ms	remaining: 17.5s
30:	learn: 0.9769268	test: 0.9777479	best: 0.9781358 (27)	total: 545ms	remaining: 17.1s
40:	learn: 0.9780931	test: 0.9780083	best: 0.9781358 (27)	total: 702ms	remaining: 16.4s
50:	learn: 0.9786601	test: 0.9782003	best: 0.9784946 (45)	total: 903ms	remaining: 16.8s
60:	learn: 0.9792766	test: 0.9781878	best: 0.9784946 (45)	total: 1.07s	remaining: 16.6s
70:	learn: 0.9797466	test: 0.9778974	best: 0.9784946 (45)	total: 1.29s	remaining: 16.9s
80:	learn: 0.9802698	test: 0.9781710	best: 0.9784946 (45)	total: 1.44s	remaining: 16.4s
90:	learn: 0.9808367	test: 0.9782609	best: 0.9784946 (45)	total: 1.68s	remaining: 16.8s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9784946237
bestIteratio

In [112]:
submission_df['prediction'] = test_class
submission_df.to_csv('submission.csv',index=False)

# Model 3

In [118]:
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat,average='micro'), True

def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1-f1_score(y_true, np.round(y_pred),average='micro')
    return 'f1_err', err

In [113]:
train_df = pd.read_csv('train_df_final.csv')
train_df = train_df.fillna(0)
test_df = pd.read_csv('test_df_final.csv')
test_df = test_df.fillna(0)
submission_df = pd.read_csv('sample_submission.csv')

In [114]:
y = train_df['label'].values

In [115]:
model_predictions_train = pd.DataFrame()
model_predictions_train['label'] = train_df['label']
model_predictions_test = pd.DataFrame()

In [117]:
features = [
    'fare_anomaly',
    'additional_fare_anomaly', 
    'duration_anomaly',
    'meter_waiting_anomaly', 
    'meter_waiting_fare_anomaly',
    'meter_waiting_till_pickup_anomaly', 
    'additional_fare_duration_anomaly',
    'additional_fare_meter_waiting_anomaly',
    'additional_fare_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_anomaly', 
    'duration_meter_waiting_fare_anomaly',
    'duration_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_anomaly',
    'meter_waiting_meter_waiting_till_pickup_anomaly',
    'meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_anomaly',
    'additional_fare_duration_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_anomaly',
    'duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
]

cat_features = [
    'fare_anomaly',
    'additional_fare_anomaly', 
    'duration_anomaly',
    'meter_waiting_anomaly', 
    'meter_waiting_fare_anomaly',
    'meter_waiting_till_pickup_anomaly', 
    'additional_fare_duration_anomaly',
    'additional_fare_meter_waiting_anomaly',
    'additional_fare_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_anomaly', 
    'duration_meter_waiting_fare_anomaly',
    'duration_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_anomaly',
    'meter_waiting_meter_waiting_till_pickup_anomaly',
    'meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_anomaly',
    'additional_fare_duration_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_anomaly',
    'duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
]

train = train_df[features]
test = test_df[features]
y = train_df['label']
for each in cat_features:
    train[each] = train[each].values.astype(int)
    test[each] = test[each].values.astype(int)
    

/home/heshan/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/heshan/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [119]:
catboost_params = {
    'loss_function':'Logloss',
    'random_state':0,
    'early_stopping_rounds':50,
    'eval_metric':'F1',
}

submission_pool = Pool(data=test_df[features], cat_features=cat_features)

train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
train_class = np.zeros(train_df.shape[0])
test_class = np.zeros(test_df.shape[0])
skf = StratifiedKFold(n_splits=3)
validation_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_pool = Pool(data=X_train, label=y_train,cat_features=cat_features)
    test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)    
    model = CatBoostClassifier(**catboost_params)
    model.fit(X=train_pool, eval_set=test_pool,verbose=10)
    train_preds[test_index] = model.predict_proba(test_pool)[:,1]
    train_class[test_index] = model.predict(test_pool)
    test_preds += model.predict_proba(submission_pool)[:,1]/3
    test_class += model.predict(submission_pool)
    validation_scores.append(f1_score(y_test,model.predict(test_pool),average='micro'))
    models.append(model)
test_class = np.where(test_class > 2, 1, 0)

Learning rate set to 0.057693
0:	learn: 0.9531844	test: 0.9530251	best: 0.9530251 (0)	total: 5.26ms	remaining: 5.26s
10:	learn: 0.9534970	test: 0.9531481	best: 0.9533160 (1)	total: 52.6ms	remaining: 4.73s
20:	learn: 0.9536295	test: 0.9533160	best: 0.9533247 (11)	total: 97.4ms	remaining: 4.54s
30:	learn: 0.9540672	test: 0.9534754	best: 0.9534754 (28)	total: 142ms	remaining: 4.43s
40:	learn: 0.9551255	test: 0.9549800	best: 0.9549800 (40)	total: 189ms	remaining: 4.43s
50:	learn: 0.9556092	test: 0.9552294	best: 0.9552294 (50)	total: 284ms	remaining: 5.28s
60:	learn: 0.9556051	test: 0.9552211	best: 0.9552294 (50)	total: 329ms	remaining: 5.07s
70:	learn: 0.9558632	test: 0.9552932	best: 0.9552932 (70)	total: 376ms	remaining: 4.92s
80:	learn: 0.9562671	test: 0.9554709	best: 0.9555597 (78)	total: 428ms	remaining: 4.86s
90:	learn: 0.9564894	test: 0.9555597	best: 0.9555597 (78)	total: 473ms	remaining: 4.73s
100:	learn: 0.9566067	test: 0.9555514	best: 0.9556485 (91)	total: 526ms	remaining: 4.68s
1

In [120]:
name = 'catboost_anomaly'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [121]:
lgb_params = {
    'objective':'binary',
    'learning_rate':0.05,
    'seed':0, 
    'metric':'f1',
    'max_depth':6
}

In [122]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_data = lgb.Dataset(X_train,y_train)
    valid_data = lgb.Dataset(X_test,y_test)
    evals_result = {}
    model = lgb.train(lgb_params, train_data,num_boost_round=1000,early_stopping_rounds=50, valid_sets=valid_data,feval=lgb_f1_score, evals_result=evals_result,verbose_eval=False)
    
    test_preds += model.predict(test) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,np.round(model.predict(X_test)),average='micro'))
    models.append(model)

In [123]:
name = 'lgb_anomaly'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [124]:
knn_params = {
    'n_neighbors':15,
    'weights':'uniform'    
}

In [125]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = KNeighborsClassifier(**knn_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    models.append(model)

In [126]:
name = 'knn_anomaly'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [127]:
rf_params = {
    'n_estimators':50,
    'max_depth':10,
    'random_state':0,    
}

In [128]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = RandomForestClassifier(**rf_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    models.append(model)


In [129]:
name = 'rf_anomaly'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [130]:
svc_params = {
    'C':2,
    'kernel':'linear',
    'random_state':0,    
    'probability': False,
    'gamma':'scale'
}

In [131]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = SVC(**svc_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    models.append(model)

In [132]:
name = 'svc_linear_anomaly'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [133]:
svc_params = {
    'C':2,
    'kernel':'rbf',
    'random_state':0,    
    'probability': False,
    'gamma':'scale'
}

In [134]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = SVC(**svc_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    models.append(model)

In [135]:
name = 'svc_rbf_anomaly'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [136]:
features = [
    'additional_fare',
    'duration',
    'meter_waiting',
    'meter_waiting_fare',
    'meter_waiting_till_pickup',
    'fare',
    'pickup_date',
    'pickup_hour',
    'pickup_minute',
    'drop_date',
    'drop_hour',
    'drop_minute',
    'pick_cluster',
    'is_more_than_one_day',
    'distance_km',
    'fare_per_km',
    'pickup_timeslot',
    'day_of_week',
    'is_weekday',
    'cal_time_difference',
    'fare_per_distance',
    'fare_per_duration',
    'avg_speed',
    'meter_waiting_per_duration',
    'meter_waiting_fare_per_meter_waiting',
    'meter_waiting_fare_per_duration',
    'addtional_fare_per_fare',
    'addtional_fare_per_distance',
    'addtional_fare_per_duration',
    'fare-additional_fare',
    'fare-additional_fare-meter_waiting_fare',
    'fare-additional_fare_per_distance',
    'fare-additional_fare_per_duration',
    'fare-additional_fare-meter_waiting_fare_per_distance',
    'fare-additional_fare-meter_waiting_fare_per_duration',
    'meter_waiting_till_pickup_per_meter_waiting',
    'meter_waiting_after_pickup',
    'meter_waiting_after_pickup_per_duration',
    'meter_waiting_till_pickup_per_duration',
    'meter_waiting_till_pickup_per_distance',
    'meter_waiting_after_pickup_per_distance',
    'meter_waiting_till_pickup_per_fare',
    'meter_waiting_after_pickup_per_fare',
    'meter_waiting_till_pickup_per_meter_waiting_fare',
    'meter_waiting_after_pickup_per_meter_waiting_fare',
    'fare_per_distance_mean',
    'fare_per_distance_mean_diff',
    'avg_speed_mean',
    'avg_speed_mean_diff',
    'meter_waiting_per_duration_mean',
    'meter_waiting_per_duration_mean_diff',
    'meter_waiting_fare_per_meter_waiting_mean',
    'meter_waiting_fare_per_meter_waiting_mean_diff',
    'meter_waiting_fare_per_duration_mean',
    'meter_waiting_fare_per_duration_mean_diff',
    'addtional_fare_per_fare_mean',
    'addtional_fare_per_fare_mean_diff',
    'addtional_fare_per_distance_mean',
    'addtional_fare_per_distance_mean_diff',
    'addtional_fare_per_duration_mean',
    'addtional_fare_per_duration_mean_diff',
]

cat_features = [
    'pickup_date',
    'pickup_hour',
    'pickup_minute',
    'drop_date',
    'drop_hour',
    'drop_minute',
    'pick_cluster',
    'is_more_than_one_day',
    'pickup_timeslot',
    'day_of_week',
    'is_weekday',
]

train = train_df[features]
test = test_df[features]
y = train_df['label']
for each in cat_features:
    train[each] = train[each].values.astype(int)
    test[each] = test[each].values.astype(int)

/home/heshan/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/heshan/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [137]:
catboost_params = {
    'loss_function':'Logloss',
    'random_state':0,
    'early_stopping_rounds':50,
    'eval_metric':'F1',
    'border_count':512
}

In [138]:
submission_pool = Pool(data=test_df[features], cat_features=cat_features)

In [139]:
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
train_class = np.zeros(train_df.shape[0])
test_class = np.zeros(test_df.shape[0])
skf = StratifiedKFold(n_splits=3)
validation_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_pool = Pool(data=X_train, label=y_train,cat_features=cat_features)
    test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)    
    model = CatBoostClassifier(**catboost_params)
    model.fit(X=train_pool, eval_set=test_pool,verbose=10)
    train_preds[test_index] = model.predict_proba(test_pool)[:,1]
    train_class[test_index] = model.predict(test_pool)
    test_preds += model.predict_proba(submission_pool)[:,1]/3
    test_class += model.predict(submission_pool)
    validation_scores.append(f1_score(y_test,model.predict(test_pool),average='micro'))
    models.append(model)
test_class = np.where(test_class > 2, 1, 0)

Learning rate set to 0.057693
0:	learn: 0.9647735	test: 0.9632415	best: 0.9632415 (0)	total: 41.4ms	remaining: 41.3s
10:	learn: 0.9700899	test: 0.9719538	best: 0.9719538 (10)	total: 346ms	remaining: 31.1s
20:	learn: 0.9726190	test: 0.9756376	best: 0.9756376 (20)	total: 576ms	remaining: 26.9s
30:	learn: 0.9750083	test: 0.9766199	best: 0.9769304 (23)	total: 831ms	remaining: 26s
40:	learn: 0.9773833	test: 0.9760903	best: 0.9769304 (23)	total: 1.13s	remaining: 26.4s
50:	learn: 0.9785424	test: 0.9761517	best: 0.9769304 (23)	total: 1.51s	remaining: 28s
60:	learn: 0.9793829	test: 0.9707952	best: 0.9769304 (23)	total: 1.86s	remaining: 28.7s
70:	learn: 0.9799435	test: 0.9690125	best: 0.9769304 (23)	total: 2.19s	remaining: 28.6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9769304099
bestIteration = 23

Shrink model to first 24 iterations.
Learning rate set to 0.057693
0:	learn: 0.9645070	test: 0.9653875	best: 0.9653875 (0)	total: 36.4ms	remaining: 36.4s
10:	learn: 0.97114

In [140]:
name = 'catboost_base'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [141]:
lgb_params = {
    'objective':'binary',
    'learning_rate':0.05,
    'seed':0, 
    'metric':'f1',
    'max_depth':6
}

In [142]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_data = lgb.Dataset(X_train,y_train)
    valid_data = lgb.Dataset(X_test,y_test)
    evals_result = {}
    model = lgb.train(lgb_params, train_data,num_boost_round=1000,early_stopping_rounds=50, valid_sets=valid_data,feval=lgb_f1_score, evals_result=evals_result,verbose_eval=False)
    
    test_preds += model.predict(test) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,np.round(model.predict(X_test)),average='micro'))
    models.append(model)

In [143]:
name = 'lgb_base'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [144]:
features = [
    'predicted_fare_diff',
    'predicted_fare_diff_per_fare',
    'predicted_fare_diff_per_predicted_fare',
    'predicted_fare_diff_per_distance',
    'predicted_duration_diff',
    'predicted_duraton_diff_per_duraton',
    'predicted_duraton_diff_per_predicted_duration',
    'predicted_duraton_diff_per_distance',
    'predicted_fare_per_duration_diff',
    'predicted_avg_speed_diff',
    'predicted_meter_waiting_diff',
    'predicted_meter_waiting_diff_per_meter_waiting',
    'predicted_meter_waiting_diff_per_distance',
    'predicted_meter_waiting_diff_per_predicted_meter_waiting',
    'predicted_meter_waiting_per_duration_diff',
    'predicted_meter_waiting_fare_diff',
    'predicted_meter_waiting_fare_diff_per_distance',
    'predicted_meter_waiting_fare_diff_per_predicted_meter_waiting_fare',
    'predicted_meter_waiting_fare_per_meter_waiting_diff',
    'predicted_meter_waiting_fare_per_duration_diff',
    'predicted_additional_fare_diff',
    'predicted_additional_fare_diff_per_additional_fare',
    'predicted_addtional_fare_diff_per_distance',
    'predicted_meter_waiting_till_pickup_diff',
    'predicted_meter_waiting_till_pickup_diff_per_meter_waiting_till_pickup',
    'predicted_meter_waiting_till_pickup_per_meter_waiting_diff',
    'predicted_fare_diff_per_distance_normalized',
    'predicted_fare_diff_normalized',
    'predicted_fare_diff_per_fare_normalized',
    'predicted_fare_diff_per_predicted_fare_normalized',
    'predicted_duraton_diff_per_duraton_normalized',
    'predicted_duraton_diff_per_predicted_duration_normalized',
    'predicted_fare_per_duration_diff_normalized',
    'predicted_avg_speed_diff_normalized',
    'predicted_meter_waiting_diff_normalized',
    'predicted_meter_waiting_diff_per_meter_waiting_normalized',
    'predicted_meter_waiting_diff_per_predicted_meter_waiting_normalized',
    'predicted_meter_waiting_per_duration_diff_normalized',
    'predicted_meter_waiting_fare_diff_normalized',
    'predicted_meter_waiting_fare_diff_per_meter_waiting_fare_normalized',
    'predicted_meter_waiting_fare_diff_per_predicted_meter_waiting_fare_normalized',
    'predicted_meter_waiting_fare_per_meter_waiting_diff_normalized',
    'predicted_meter_waiting_fare_per_duration_diff_normalized',
    'predicted_additional_fare_diff_normalized',
    'predicted_additional_fare_diff_per_additional_fare_normalized',
]

cat_features = []

train = train_df[features]
test = test_df[features]
y = train_df['label']
for each in cat_features:
    train[each] = train[each].values.astype(int)
    test[each] = test[each].values.astype(int)

In [145]:
catboost_params = {
    'loss_function':'Logloss',
    'random_state':0,
    'early_stopping_rounds':50,
    'eval_metric':'F1',
    'border_count':512
}

In [146]:
submission_pool = Pool(data=test_df[features], cat_features=cat_features)

In [147]:
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
train_class = np.zeros(train_df.shape[0])
test_class = np.zeros(test_df.shape[0])
skf = StratifiedKFold(n_splits=3)
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_pool = Pool(data=X_train, label=y_train,cat_features=cat_features)
    test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)    
    model = CatBoostClassifier(**catboost_params)
    model.fit(X=train_pool, eval_set=test_pool,verbose=10)
    train_preds[test_index] = model.predict_proba(test_pool)[:,1]
    train_class[test_index] = model.predict(test_pool)
    test_preds += model.predict_proba(submission_pool)[:,1]/3
    test_class += model.predict(submission_pool)
    validation_scores.append(f1_score(y_test,model.predict(test_pool),average='micro'))
    models.append(model)
test_class = np.where(test_class > 2, 1, 0)

Learning rate set to 0.057693
0:	learn: 0.9691234	test: 0.9694540	best: 0.9694540 (0)	total: 22ms	remaining: 22s
10:	learn: 0.9729214	test: 0.9740508	best: 0.9740508 (10)	total: 227ms	remaining: 20.4s
20:	learn: 0.9743174	test: 0.9757170	best: 0.9757170 (20)	total: 364ms	remaining: 17s
30:	learn: 0.9751363	test: 0.9761563	best: 0.9761563 (30)	total: 515ms	remaining: 16.1s
40:	learn: 0.9760651	test: 0.9764097	best: 0.9765078 (36)	total: 650ms	remaining: 15.2s
50:	learn: 0.9767152	test: 0.9770479	best: 0.9771418 (49)	total: 837ms	remaining: 15.6s
60:	learn: 0.9772836	test: 0.9774147	best: 0.9778034 (58)	total: 996ms	remaining: 15.3s
70:	learn: 0.9779419	test: 0.9776026	best: 0.9778034 (58)	total: 1.18s	remaining: 15.5s
80:	learn: 0.9782275	test: 0.9780643	best: 0.9780643 (79)	total: 1.39s	remaining: 15.8s
90:	learn: 0.9786458	test: 0.9784491	best: 0.9784491 (88)	total: 1.59s	remaining: 15.9s
100:	learn: 0.9793037	test: 0.9785433	best: 0.9785433 (94)	total: 1.8s	remaining: 16s
110:	learn:

In [148]:
name = 'catboost_pred_diff'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [149]:
lgb_params = {
    'objective':'binary',
    'learning_rate':0.05,
    'seed':0, 
    'metric':'f1',
    'max_depth':6
}

In [150]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_data = lgb.Dataset(X_train,y_train)
    valid_data = lgb.Dataset(X_test,y_test)
    evals_result = {}
    model = lgb.train(lgb_params, train_data,num_boost_round=1000,early_stopping_rounds=50, valid_sets=valid_data,feval=lgb_f1_score, evals_result=evals_result,verbose_eval=False)
    
    test_preds += model.predict(test) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,np.round(model.predict(X_test)),average='micro'))
    models.append(model)


In [151]:
name = 'lgb_pred_diff'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [152]:
knn_params = {
    'n_neighbors':10,
    'weights':'uniform'    
}

In [153]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = KNeighborsClassifier(**knn_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    models.append(model)

In [154]:
name = 'knn_pred_diff'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [155]:
svc_params = {
    'C':1,
    'kernel':'rbf',
    'random_state':0,    
    'probability': False,
    'gamma':'auto'
}

In [156]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = SVC(**svc_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    models.append(model)

In [157]:
name = 'svm_pred_diff'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [159]:
features = [
    'predicted_fare_diff_per_fare',
    'predicted_fare_diff_per_predicted_fare',
    'predicted_fare_diff_per_distance',
    'predicted_duraton_diff_per_duraton',
    'predicted_duraton_diff_per_predicted_duration',
    'predicted_duraton_diff_per_distance',
    'predicted_fare_per_duration_diff',
    'predicted_meter_waiting_diff_per_meter_waiting',
    'predicted_meter_waiting_diff_per_distance',
    'predicted_meter_waiting_diff_per_predicted_meter_waiting',
    'predicted_meter_waiting_per_duration_diff',
    'predicted_meter_waiting_fare_diff_per_distance',
    'predicted_meter_waiting_fare_diff_per_predicted_meter_waiting_fare',
    'predicted_meter_waiting_fare_per_meter_waiting_diff',
    'predicted_meter_waiting_fare_per_duration_diff',
    'predicted_additional_fare_diff_per_additional_fare',
    'predicted_addtional_fare_diff_per_distance',
    'predicted_meter_waiting_till_pickup_diff_per_meter_waiting_till_pickup',
    'predicted_meter_waiting_till_pickup_per_meter_waiting_diff'
]

cat_features = []

train = train_df[features]
test = test_df[features]
y = train_df['label']
for each in cat_features:
    train[each] = train[each].values.astype(int)
    test[each] = test[each].values.astype(int)

In [160]:
catboost_params = {
    'loss_function':'Logloss',
    'random_state':0,
    'early_stopping_rounds':50,
    'eval_metric':'F1',
    'border_count':512
}

In [161]:
submission_pool = Pool(data=test_df[features], cat_features=cat_features)

In [162]:
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
train_class = np.zeros(train_df.shape[0])
test_class = np.zeros(test_df.shape[0])
skf = StratifiedKFold(n_splits=3)
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_pool = Pool(data=X_train, label=y_train,cat_features=cat_features)
    test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)    
    model = CatBoostClassifier(**catboost_params)
    model.fit(X=train_pool, eval_set=test_pool,verbose=10)
    train_preds[test_index] = model.predict_proba(test_pool)[:,1]
    train_class[test_index] = model.predict(test_pool)
    test_preds += model.predict_proba(submission_pool)[:,1]/3
    test_class += model.predict(submission_pool)
    validation_scores.append(f1_score(y_test,model.predict(test_pool),average='micro'))
    models.append(model)
test_class = np.where(test_class > 2, 1, 0)

Learning rate set to 0.057693
0:	learn: 0.9645377	test: 0.9632381	best: 0.9632381 (0)	total: 18.9ms	remaining: 18.9s
10:	learn: 0.9730582	test: 0.9747867	best: 0.9747867 (10)	total: 174ms	remaining: 15.7s
20:	learn: 0.9741948	test: 0.9761905	best: 0.9761905 (20)	total: 296ms	remaining: 13.8s
30:	learn: 0.9750706	test: 0.9772269	best: 0.9772269 (30)	total: 374ms	remaining: 11.7s
40:	learn: 0.9756728	test: 0.9773121	best: 0.9774190 (31)	total: 453ms	remaining: 10.6s
50:	learn: 0.9764790	test: 0.9777906	best: 0.9777949 (46)	total: 553ms	remaining: 10.3s
60:	learn: 0.9769448	test: 0.9777821	best: 0.9777949 (46)	total: 637ms	remaining: 9.8s
70:	learn: 0.9777075	test: 0.9777778	best: 0.9778761 (68)	total: 718ms	remaining: 9.39s
80:	learn: 0.9784766	test: 0.9778718	best: 0.9781584 (74)	total: 817ms	remaining: 9.27s
90:	learn: 0.9788560	test: 0.9783508	best: 0.9783508 (89)	total: 907ms	remaining: 9.05s
100:	learn: 0.9793686	test: 0.9781500	best: 0.9783508 (89)	total: 993ms	remaining: 8.84s
110

In [163]:
name = 'catboost_unit_diff'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [164]:
lgb_params = {
    'objective':'binary',
    'learning_rate':0.05,
    'seed':0, 
    'metric':'f1',
    'max_depth':8
}

In [165]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_data = lgb.Dataset(X_train,y_train)
    valid_data = lgb.Dataset(X_test,y_test)
    evals_result = {}
    model = lgb.train(lgb_params, train_data,num_boost_round=1000,early_stopping_rounds=50, valid_sets=valid_data,feval=lgb_f1_score, evals_result=evals_result,verbose_eval=False)
    
    test_preds += model.predict(test) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,np.round(model.predict(X_test)),average='micro'))
    models.append(model)

In [166]:
name = 'lgb_unit_diff'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [167]:
knn_params = {
    'n_neighbors':10,
    'weights':'distance'    
}

In [168]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = KNeighborsClassifier(**knn_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    models.append(model)

In [169]:
name = 'knn_unit_diff'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [170]:
svc_params = {
    'C':5,
    'kernel':'rbf',
    'random_state':0,    
    'probability': False,
    'gamma':'auto'
}

In [171]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = SVC(**svc_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    models.append(model)

In [172]:
name = 'rbf_svm_unit_diff'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [173]:
features = [
    'predicted_fare_diff_per_distance_normalized',
    'predicted_fare_diff_normalized',
    'predicted_fare_diff_per_fare_normalized',
    'predicted_fare_diff_per_predicted_fare_normalized',
    'predicted_duraton_diff_per_duraton_normalized',
    'predicted_duraton_diff_per_predicted_duration_normalized',
    'predicted_fare_per_duration_diff_normalized',
    'predicted_avg_speed_diff_normalized',
    'predicted_meter_waiting_diff_normalized',
    'predicted_meter_waiting_diff_per_meter_waiting_normalized',
    'predicted_meter_waiting_diff_per_predicted_meter_waiting_normalized',
    'predicted_meter_waiting_per_duration_diff_normalized',
    'predicted_meter_waiting_fare_diff_normalized',
    'predicted_meter_waiting_fare_diff_per_meter_waiting_fare_normalized',
    'predicted_meter_waiting_fare_diff_per_predicted_meter_waiting_fare_normalized',
    'predicted_meter_waiting_fare_per_meter_waiting_diff_normalized',
    'predicted_meter_waiting_fare_per_duration_diff_normalized',
    'predicted_additional_fare_diff_normalized',
    'predicted_additional_fare_diff_per_additional_fare_normalized',
]

cat_features = []

train = train_df[features]
test = test_df[features]
y = train_df['label']
for each in cat_features:
    train[each] = train[each].values.astype(int)
    test[each] = test[each].values.astype(int)

In [174]:
knn_params = {
    'n_neighbors':15,
    'weights':'distance'    
}

In [175]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = KNeighborsClassifier(**knn_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    org_scores.append(f1_score(y_org,model.predict(train_org.values),average='micro'))
    models.append(model)

NameError: name 'y_org' is not defined

In [ ]:
name = 'knn_unit_diff_norm'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [ ]:
svc_params = {
    'C':5,
    'kernel':'rbf',
    'random_state':0,    
    'probability': False,
    'gamma':'scale'
}

In [ ]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = SVC(**svc_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    models.append(model)

In [ ]:
name = 'svm_rbf_unit_diff_norm'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [176]:
features = [
    'fare_per_distance',
    'fare_per_duration',
    'avg_speed',
    'meter_waiting_per_duration',
    'meter_waiting_fare_per_meter_waiting',
    'meter_waiting_fare_per_duration',
    'addtional_fare_per_fare',
    'addtional_fare_per_distance',
    'addtional_fare_per_duration',
    'fare-additional_fare_per_distance',
    'fare-additional_fare_per_duration',
    'fare-additional_fare-meter_waiting_fare_per_distance',
    'fare-additional_fare-meter_waiting_fare_per_duration',
    'meter_waiting_till_pickup_per_meter_waiting',
    'meter_waiting_after_pickup_per_duration',
    'meter_waiting_till_pickup_per_duration',
    'meter_waiting_till_pickup_per_distance',
    'meter_waiting_after_pickup_per_distance',
    'meter_waiting_till_pickup_per_fare',
    'meter_waiting_after_pickup_per_fare',
    'meter_waiting_till_pickup_per_meter_waiting_fare',
    'meter_waiting_after_pickup_per_meter_waiting_fare',    
]

cat_features = []

train = train_df[features]
test = test_df[features]
y = train_df['label']
for each in cat_features:
    train[each] = train[each].values.astype(int)
    test[each] = test[each].values.astype(int)

In [177]:
catboost_params = {
    'loss_function':'Logloss',
    'random_state':0,
    'early_stopping_rounds':50,
    'eval_metric':'F1',
    'border_count':512
}

In [178]:
submission_pool = Pool(data=test_df[features], cat_features=cat_features)

In [179]:
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
train_class = np.zeros(train_df.shape[0])
test_class = np.zeros(test_df.shape[0])
skf = StratifiedKFold(n_splits=3)
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_pool = Pool(data=X_train, label=y_train,cat_features=cat_features)
    test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)    
    model = CatBoostClassifier(**catboost_params)
    model.fit(X=train_pool, eval_set=test_pool,verbose=10)
    train_preds[test_index] = model.predict_proba(test_pool)[:,1]
    train_class[test_index] = model.predict(test_pool)
    test_preds += model.predict_proba(submission_pool)[:,1]/3
    test_class += model.predict(submission_pool)
    validation_scores.append(f1_score(y_test,model.predict(test_pool),average='micro'))
    models.append(model)
test_class = np.where(test_class > 2, 1, 0)

Learning rate set to 0.057693
0:	learn: 0.9637438	test: 0.9634802	best: 0.9634802 (0)	total: 20.4ms	remaining: 20.4s
10:	learn: 0.9706593	test: 0.9747052	best: 0.9749786 (9)	total: 175ms	remaining: 15.7s
20:	learn: 0.9737268	test: 0.9762337	best: 0.9763359 (19)	total: 279ms	remaining: 13s
30:	learn: 0.9750238	test: 0.9774249	best: 0.9774249 (30)	total: 362ms	remaining: 11.3s
40:	learn: 0.9757680	test: 0.9782713	best: 0.9783691 (39)	total: 444ms	remaining: 10.4s
50:	learn: 0.9766909	test: 0.9786337	best: 0.9787397 (46)	total: 548ms	remaining: 10.2s
60:	learn: 0.9773466	test: 0.9784297	best: 0.9787397 (46)	total: 646ms	remaining: 9.95s
70:	learn: 0.9778670	test: 0.9786214	best: 0.9787397 (46)	total: 740ms	remaining: 9.68s
80:	learn: 0.9782390	test: 0.9787152	best: 0.9789110 (77)	total: 860ms	remaining: 9.76s
90:	learn: 0.9785625	test: 0.9787071	best: 0.9790049 (82)	total: 966ms	remaining: 9.65s
100:	learn: 0.9789821	test: 0.9782171	best: 0.9790049 (82)	total: 1.09s	remaining: 9.72s
110:	

In [180]:
name = 'catboost_unit'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [181]:
lgb_params = {
    'objective':'binary',
    'learning_rate':0.05,
    'seed':0, 
    'metric':'f1',
    'max_depth':8
}

In [182]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    train_data = lgb.Dataset(X_train,y_train)
    valid_data = lgb.Dataset(X_test,y_test)
    evals_result = {}
    model = lgb.train(lgb_params, train_data,num_boost_round=1000,early_stopping_rounds=50, valid_sets=valid_data,feval=lgb_f1_score, evals_result=evals_result,verbose_eval=False)
    
    test_preds += model.predict(test) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,np.round(model.predict(X_test)),average='micro'))
    models.append(model)

In [183]:
name = 'lgb_unit'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [184]:
knn_params = {
    'n_neighbors':10,
    'weights':'distance'    
}

In [185]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = KNeighborsClassifier(**knn_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    models.append(model)

In [186]:
name = 'knn_unit'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [187]:
svc_params = {
    'C':10,
    'kernel':'rbf',
    'random_state':0,    
    'probability': False,
    'gamma':'auto'
}

In [188]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
org_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = SVC(**svc_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    models.append(model)

In [189]:
name = 'svm_unit'
model_predictions_train[name] = train_preds
model_predictions_test[name] = test_preds

In [190]:
for col in model_predictions_test.columns:
    model_predictions_train[col] = np.round(model_predictions_train[col]).astype(int)
    model_predictions_test[col] = np.round(model_predictions_test[col]).astype(int)

In [191]:
features = [
    'catboost_unit',
    'knn_pred_diff',
    'catboost_pred_diff',
    'lgb_pred_diff',
    'catboost_unit_diff',
    'lgb_unit_diff',
    'svm_pred_diff',
    'svc_linear_anomaly',
    'catboost_anomaly',
    'rbf_svm_unit_diff',
    'rf_anomaly'
]
cat_cols = [
    'knn_pred_diff',
    'svm_pred_diff',
    'svc_linear_anomaly',
    'svc_linear_anomaly',
    'rbf_svm_unit_diff',
    'rf_anomaly',
]

In [192]:
for each in cat_cols:
    model_predictions_train[each] = model_predictions_train[each].astype(int)
    model_predictions_test[each] = model_predictions_test[each].astype(int)

In [193]:
train = model_predictions_train[features]
test = model_predictions_test[features]
y = train_df['label']

In [194]:
logistic_reg_params = {
    'penalty':'l2',
    'C':1.0,
    'solver':'lbfgs',
    'max_iter':100
}

In [195]:
folds = 3
skf = StratifiedKFold(n_splits=folds)
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
validation_scores = []
models = []
for train_index, test_index in skf.split(train, y):
    X_train, X_test = train.iloc[train_index,:].values, train.iloc[test_index,:].values
    y_train, y_test = y[train_index], y[test_index]
    
    model = LogisticRegression(**logistic_reg_params)
    model.fit(X_train, y_train)
    
    test_preds += model.predict(test.values) / 3
    train_preds[test_index] = model.predict(X_test)
    validation_scores.append(f1_score(y_test,model.predict(X_test),average='micro'))
    models.append(model)

In [196]:
submission_df['prediction'] = np.round(test_preds).astype(int)
submission_df.to_csv('submission.csv',index=False)